In [30]:
import cv2
import zipfile
import numpy as np
import pandas as pd
from imutils import paths
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

Разархивирование папки с изображениями

In [10]:
with zipfile.ZipFile('train_task.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [16]:
imagePaths = sorted(list(paths.list_images('train')))

Получение гистограмы для каждого изображения и кодировка классов

In [12]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [18]:
X = []
y = []
for image in imagePaths:
  X.append(extract_histogram(cv2.imread(image)))
  if 'cat' in image:
    y.append(0)
  else:
    y.append(1)

In [19]:
X = pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.242645,0.073122,0.000232,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.087363,...,0.00000,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000
1,0.418818,0.126781,0.000085,0.000000,0.000000,0.0,0.0,0.0,0.005085,0.076441,...,0.00000,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000042,0.000000
2,0.356737,0.184168,0.020616,0.001619,0.000717,0.0,0.0,0.0,0.000041,0.027440,...,0.00002,0.000000,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.000000,0.007726
3,0.001266,0.019995,0.011941,0.001583,0.000158,0.0,0.0,0.0,0.000000,0.002145,...,0.01595,0.002233,0.0,0.000000,0.0000,0.00000,0.000000,0.000000,0.023354,0.497077
4,0.000000,0.000105,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000366,...,0.00000,0.000000,0.0,0.002719,0.7748,0.34338,0.038583,0.114442,0.126833,0.008469


Разделение данных на тренировочные и тестовые

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=19)

Обучение модели

In [63]:
model = LinearSVC(C = 0.6, random_state = 19).fit(X_train, y_train)
model.classes_

array([0, 1])

**Значение коэффициента Тетта для 11, 13 и 318**

In [38]:
print(round(model.coef_[0, 11-1], 2))
print(round(model.coef_[0, 13-1], 2))
print(round(model.coef_[0, 318-1], 2))

-0.42
-0.08
-0.09


**Значение метрики F1**

In [39]:
round(f1_score(y_test, model.predict(X_test), average='macro'), 2)

0.6

**Предсказание класса для изображений**

In [64]:
model.predict([extract_histogram(cv2.imread('dog.1035.jpg'))])

array([0])

In [65]:
model.predict([extract_histogram(cv2.imread('dog.1022.jpg'))])

array([0])

In [66]:
model.predict([extract_histogram(cv2.imread('cat.1018.jpg'))])

array([0])

In [67]:
model.predict([extract_histogram(cv2.imread('cat.1002.jpg'))])

array([0])